In [1]:
# 해당 유저의 최근 매치 5개 조회
import requests, json
from tqdm import tqdm
from server import *

from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

def collect_match():

    with open("data/user_ouid.json", "r", encoding="utf-8") as file:
    
        user = json.load(file)  # JSON 데이터를 파이썬 리스트로 변환

    total_match_list = []

    func = USER_FUNCTION['match']

    matchtype = 50 # 공식 경기 1vs1
    offset = 0
    limit = 5

    for nickname,id in tqdm(user.items()):

        urlString = basic_link + func + "?ouid=" + id + "&matchtype=" + str(matchtype) + "&offset=" + str(offset) + "&limit=" + str(limit)

        response = requests.get(urlString, headers = headers)
        
        result = response.json()

        for match_id in result:

            total_match_list.append(match_id)

    # JSON 파일로 저장
    with open("data/matchid.json", "w", encoding="utf-8") as file:
        
        json.dump(list(set(total_match_list)), file, ensure_ascii=False, indent=4)  # ensure_ascii=False로 한글 깨짐 방지

# 기본 설정
default_args = {
    'owner': 'user',
    'depends_on_past': False,
    'start_date': datetime(2025, 1, 17),
    'retries': 1,
}

# DAG 정의
dag = DAG(
    dag_id='web_crawling_dag',
    default_args=default_args,
    description='A DAG for web crawling',
    schedule_interval='@monthly',  # 매달 1회 실행
    catchup=False,
)

# PythonOperator로 작업 등록
crawl_task = PythonOperator(
    task_id='crawl_data_task',
    python_callable=collect_match,  # 크롤링 함수 연결
    dag=dag,
)

#  DAG 실행 순서 정의
crawl_task  # 단일 작업일 경우 순서 지정 생략 가능

/var/folders/wg/k0b4bxhx74zcv6t4ww39dh000000gn/T/ipykernel_30494/1741280495.py:7 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

/var/folders/wg/k0b4bxhx74zcv6t4ww39dh000000gn/T/ipykernel_30494/1741280495.py:50 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): crawl_data_task>